## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-10-12-11-46-04 +0000,bbc,Afghan Taliban says Pakistani troops killed in...,https://www.bbc.com/news/articles/cgmzmn4971zo...
1,2025-10-12-11-20-48 +0000,nyt,Rethinking Bruce Springsteen’s Difficult Years,https://www.nytimes.com/2025/10/12/us/bruce-sp...
2,2025-10-12-11-10-02 +0000,nyt,Overnight Violence Between Afghanistan and Pak...,https://www.nytimes.com/2025/10/12/world/asia/...
3,2025-10-12-11-04-00 +0000,wsj,Hamas Says It’s Ready to Hand Over 20 Living I...,https://www.wsj.com/world/middle-east/hamas-sa...
4,2025-10-12-11-03-57 +0000,bbc,Nuns escape fire at historic Italian monastery,https://www.bbc.com/news/videos/c1jzj1n797yo?a...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2360/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
149,trump,32
116,gaza,16
59,shutdown,13
360,tennessee,9
37,no,8


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
137,2025-10-11-20-36-00 +0000,wsj,Trump said funds have been identified to pay m...,https://www.wsj.com/politics/policy/trump-says...,85
158,2025-10-11-18-12-00 +0000,nypost,President Trump will make sure military troops...,https://nypost.com/2025/10/11/us-news/presiden...,85
164,2025-10-11-17-17-44 +0000,nypost,Putin chides Nobel winners who ‘did nothing fo...,https://nypost.com/2025/10/11/world-news/putin...,84
61,2025-10-12-04-37-28 +0000,bbc,Israelis praise Trump at huge rally ahead of e...,https://www.bbc.com/news/articles/c75q5qng0rgo...,79
200,2025-10-11-12-30-00 +0000,nypost,First hostage freed from Hamas’ grasp thanks T...,https://nypost.com/2025/10/11/world-news/first...,74


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
137,85,2025-10-11-20-36-00 +0000,wsj,Trump said funds have been identified to pay m...,https://www.wsj.com/politics/policy/trump-says...
164,52,2025-10-11-17-17-44 +0000,nypost,Putin chides Nobel winners who ‘did nothing fo...,https://nypost.com/2025/10/11/world-news/putin...
161,44,2025-10-11-17-33-00 +0000,wsj,Authorities Find No Survivors After Deadly Bla...,https://www.wsj.com/us-news/tennessee-explosio...
50,36,2025-10-12-07-22-45 +0000,nypost,"Chinese coast guard fires water cannon, rams P...",https://nypost.com/2025/10/12/world-news/chine...
200,34,2025-10-11-12-30-00 +0000,nypost,First hostage freed from Hamas’ grasp thanks T...,https://nypost.com/2025/10/11/world-news/first...
111,29,2025-10-11-22-45-15 +0000,nypost,"Argentinian model, singer Fede Dorcaz shot and...",https://nypost.com/2025/10/11/world-news/argen...
125,29,2025-10-11-21-29-39 +0000,startribune,St. Paul police: 22-year-old man was assaulted...,https://www.startribune.com/st-paul-police-man...
190,29,2025-10-11-13-44-51 +0000,nypost,Joe Biden undergoing radiation in new phase of...,https://nypost.com/2025/10/11/us-news/joe-bide...
76,28,2025-10-12-02-33-57 +0000,nypost,Tech billionaire Mark Benioff calls for Trump ...,https://nypost.com/2025/10/11/us-news/time-mag...
196,27,2025-10-11-12-55-02 +0000,nypost,AOC tries to squeeze campaign cash off Trump’s...,https://nypost.com/2025/10/11/us-news/aoc-trie...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
